# Homework 8 Question 1: Voting
Governor Blue of the state of Berry is attempting to get the state legislator to
gerrymander Berry’s congressional districts. The state consists of ten cities, and the numbers of
registered Republicans and Democrats (in thousands) in each city are shown below 

City | Republicans | Democrats
--- | --- | ---
1 | 80 | 34
2 | 60 | 44
3 | 40 | 44
4 | 20 | 24
5 | 40 | 114
6 | 40 | 64
7 | 70 | 14
8 | 50 | 44
9 | 70 | 54
10 | 70 | 64

Berry has five congressional representatives. To form the five congressional districts, cities must be
grouped together according to the following restrictions:

- Districts cannot subdivide cities; all voters in a city must be in the same district.
-  Each district must contain between 150,000 and 250,000 voters (there are no independent voters).

Governor Blue is a Democrat. Assume 100% voter turnout and that each voter always votes according
to their registered party. Formulate and solve an optimization problem to help Governor Blue maximize
the number of congressional districts that have a Democratic majority.

## Problem Data 

In [1]:
cities = [:one, :two, :three, :four, :five, :six, :seven, :eight, :nine, :ten]
raw_repub = [80, 60, 40, 20, 40, 40, 70, 50, 70, 70]'
republicans = Dict(zip(cities, raw_repub))
raw_demo = [34, 44, 44, 24, 114, 64, 14, 44, 54, 64]'
democrats = Dict(zip(cities, raw_demo))
n_cities = length(cities)
n_districts = 5;

## Problem Model 

In [2]:
using JuMP, Cbc

m = Model(solver=CbcSolver())

@variable(m, x[1:n_cities, 1:n_districts], Bin)
@variable(m, majority[1:n_districts], Bin)

@constraint(m, cityToOneDistrict[i=1:n_cities], sum(x[i,j] for j in 1:n_districts) == 1)

@constraint(m, votersMin[j=1:n_districts], sum(x[i,j]*
                    (democrats[cities[i]] + republicans[cities[i]])
                    for i in 1:n_cities) >= 150)
@constraint(m, votersMax[j=1:n_districts], sum(x[i,j]*
                                (democrats[cities[i]] + republicans[cities[i]])
                    for i in 1:n_cities) <= 250)
                 
# If There is majority then # of democrats are more than # of republicans
@constraint(m, majorityConstr[j=1:n_districts], sum(x[i,j]*
                                            ( republicans[cities[i]] - democrats[cities[i]])
                    for i in 1:n_cities) <= 250(1-majority[j]))
@objective(m, Max, sum(majority))
m

Maximization problem with:
 * 25 linear constraints
 * 55 variables: 55 binary
Solver is CbcMathProg

In [3]:
status = solve(m)
println("Status: ",status)
println("Majority in districts ", getvalue(majority))
assignment = getvalue(x)
println("Assignment of cities to districts")
[println(assignment[:,j]) for j in 1:n_districts]
println("# of Voters per district, # of Democrats, # of Republicans")
[println(raw_demo*assignment[:,j] + raw_repub*assignment[:,j],"\t",
        (raw_demo*assignment[:,j])[1],"\t",
        (raw_repub*assignment[:,j])[1])
    for j in 1:n_districts];

Status: Optimal
Majority in districts [1.0,0.0,0.0,1.0,1.0]
Assignment of cities to districts
[0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]
[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]
[0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]
[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0]
# of Voters per district, # of Democrats, # of Republicans
[222.0]	112.0	110.0
[238.0]	88.0	150.0
[188.0]	58.0	130.0
[154.0]	114.0	40.0
[238.0]	128.0	110.0
